In [3]:
import numpy as np
import pandas as pd
import json
import pprint
import json
pp = pprint.PrettyPrinter(indent=4)

## ML-100k - 0.7 train / 0.3 test - sparse item

In [4]:
results = [
'../metrics/mfnn/2016-05-27_20.32.35_mfnn_ml-100k_e20_tt-0.7_task-19_user-metrics.csv',
'../metrics/mfnn/2016-05-27_20.33.16_mfnn_ml-100k_e20_tt-0.7_task-6_user-metrics.csv',
'../metrics/mfnn/2016-05-27_20.40.31_mfnn_ml-100k_e20_tt-0.7_task-8_user-metrics.csv',
'../metrics/mfnn/2016-05-27_21.23.18_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv',
'../metrics/mfnn/2016-05-27_21.36.57_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-05-27_22.01.54_mfnn_ml-100k_e20_tt-0.7_task-12_user-metrics.csv',
'../metrics/mfnn/2016-05-27_22.07.28_mfnn_ml-100k_e20_tt-0.7_task-5_user-metrics.csv',
'../metrics/mfnn/2016-05-27_22.14.10_mfnn_ml-100k_e20_tt-0.7_task-9_user-metrics.csv',
'../metrics/mfnn/2016-05-27_23.32.04_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-05-28_00.07.22_mfnn_ml-100k_e20_tt-0.7_task-7_user-metrics.csv',
'../metrics/mfnn/2016-05-28_00.26.03_mfnn_ml-100k_e20_tt-0.7_task-10_user-metrics.csv',
'../metrics/mfnn/2016-05-28_01.26.12_mfnn_ml-100k_e20_tt-0.7_task-11_user-metrics.csv',
'../metrics/mfnn/2016-05-28_02.29.58_mfnn_ml-100k_e20_tt-0.7_task-23_user-metrics.csv',
'../metrics/mfnn/2016-05-28_02.32.17_mfnn_ml-100k_e20_tt-0.7_task-18_user-metrics.csv',
'../metrics/mfnn/2016-05-28_03.04.40_mfnn_ml-100k_e20_tt-0.7_task-17_user-metrics.csv',
'../metrics/mfnn/2016-05-28_04.07.27_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
'../metrics/mfnn/2016-05-30_17.43.10_mfnn_ml-100k_e20_tt-0.7_task-23_user-metrics.csv',
'../metrics/mfnn/2016-05-30_17.44.44_mfnn_ml-100k_e20_tt-0.7_task-7_user-metrics.csv',
'../metrics/mfnn/2016-05-30_17.45.10_mfnn_ml-100k_e20_tt-0.7_task-15_user-metrics.csv',
'../metrics/mfnn/2016-05-30_17.49.49_mfnn_ml-100k_e20_tt-0.7_task-3_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.03.38_mfnn_ml-100k_e20_tt-0.7_task-12_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.09.38_mfnn_ml-100k_e20_tt-0.7_task-19_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.23.41_mfnn_ml-100k_e20_tt-0.7_task-4_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.27.02_mfnn_ml-100k_e20_tt-0.7_task-2_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.34.38_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.35.21_mfnn_ml-100k_e20_tt-0.7_task-16_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.35.26_mfnn_ml-100k_e20_tt-0.7_task-11_user-metrics.csv',
'../metrics/mfnn/2016-05-30_18.55.22_mfnn_ml-100k_e20_tt-0.7_task-8_user-metrics.csv',
'../metrics/mfnn/2016-05-30_19.03.48_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
'../metrics/mfnn/2016-05-30_19.15.45_mfnn_ml-100k_e20_tt-0.7_task-6_user-metrics.csv',
'../metrics/mfnn/2016-05-30_19.20.26_mfnn_ml-100k_e20_tt-0.7_task-5_user-metrics.csv',
'../metrics/mfnn/2016-05-30_20.39.37_mfnn_ml-100k_e20_tt-0.7_task-17_user-metrics.csv',
'../metrics/mfnn/2016-05-31_02.58.20_mfnn_ml-100k_e20_tt-0.7_task-9_user-metrics.csv',
'../metrics/mfnn/2016-05-31_05.02.41_mfnn_ml-100k_e20_tt-0.7_task-22_user-metrics.csv',
'../metrics/mfnn/2016-05-31_07.04.38_mfnn_ml-100k_e20_tt-0.7_task-18_user-metrics.csv',
'../metrics/mfnn/2016-05-31_10.53.16_mfnn_ml-100k_e20_tt-0.7_task-10_user-metrics.csv',
'../metrics/mfnn/2016-05-31_11.40.24_mfnn_ml-100k_e20_tt-0.7_task-14_user-metrics.csv',
]

In [5]:
result_map = map(lambda p: (p, pd.read_csv(p)['auc'].mean()), results)
sorted(result_map, key=lambda t:t[1], reverse=True)

[('../metrics/mfnn/2016-05-31_10.53.16_mfnn_ml-100k_e20_tt-0.7_task-10_user-metrics.csv',
  0.9344515580215218),
 ('../metrics/mfnn/2016-05-30_19.15.45_mfnn_ml-100k_e20_tt-0.7_task-6_user-metrics.csv',
  0.9329419457883121),
 ('../metrics/mfnn/2016-05-31_07.04.38_mfnn_ml-100k_e20_tt-0.7_task-18_user-metrics.csv',
  0.9327237853915465),
 ('../metrics/mfnn/2016-05-27_22.14.10_mfnn_ml-100k_e20_tt-0.7_task-9_user-metrics.csv',
  0.932350660340112),
 ('../metrics/mfnn/2016-05-30_18.34.38_mfnn_ml-100k_e20_tt-0.7_task-0_user-metrics.csv',
  0.9313797058717969),
 ('../metrics/mfnn/2016-05-30_19.03.48_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
  0.9283845996374857),
 ('../metrics/mfnn/2016-05-27_23.32.04_mfnn_ml-100k_e20_tt-0.7_task-1_user-metrics.csv',
  0.9248770143862938),
 ('../metrics/mfnn/2016-05-27_20.33.16_mfnn_ml-100k_e20_tt-0.7_task-6_user-metrics.csv',
  0.9243492483928197),
 ('../metrics/mfnn/2016-05-28_02.32.17_mfnn_ml-100k_e20_tt-0.7_task-18_user-metrics.csv',
  0.923276245

In [6]:
# best config
with open('../models/mfnn/2016-05-31_10.53.16_mfnn_ml-100k_e20_tt-0.7_task-10_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': False,
    u'experiment_name': u'mfnn_ml-100k_e20_tt-0.7_task-10',
    u'hit_threshold': 4,
    u'lr': 0.03,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 96,
    u'nb_movies': 1261,
    u'nb_users': 943,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-100k/ratings.csv',
    u'reg_lambda': 0.01,
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-100k/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-100k/sparse-item/0.7-train.csv',
    u'train_tes

In [7]:
# second best config
with open('../models/mfnn/2016-05-30_19.15.45_mfnn_ml-100k_e20_tt-0.7_task-6_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': False,
    u'experiment_name': u'mfnn_ml-100k_e20_tt-0.7_task-6',
    u'hit_threshold': 4,
    u'lr': 0.03,
    u'lr_decay': 0.0005,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 96,
    u'nb_movies': 1261,
    u'nb_users': 943,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-100k/ratings.csv',
    u'reg_lambda': 0.01,
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-100k/sparse-item/0.7-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-100k/sparse-item/0.7-train.csv',
    u'train_te

In [8]:
best_metrics = pd.read_csv('../metrics/mfnn/2016-05-31_10.53.16_mfnn_ml-100k_e20_tt-0.7_task-10_user-metrics.csv')
best_metrics.mean(0)

auc                          0.934452
avg_precision                0.318685
f1                           0.278498
fcp                          0.715248
nb_movies_not_in_train    1193.269353
nb_test_movies              28.958643
nb_train_movies             67.730647
precision_at_20              0.275928
recall_at_20                 0.424804
reciprocal_rank              0.644229
spearman_rank_corr           0.191657
user_id                    472.000000
dtype: float64

## ML-1M - 0.7 train / 0.3 test - sparse item

In [ ]:
results = [

]

In [ ]:
result_map = map(lambda p: (p, pd.read_csv(p)['auc'].mean()), results)
sorted(result_map, key=lambda t:t[1], reverse=True)

In [ ]:
# best config
with open('../models/mfnn/_config.json') as f:
    pp.pprint(json.loads(f.read()))

In [ ]:
best_metrics = pd.read_csv('../metrics/mfnn/_user-metrics.csv')
best_metrics.mean(0)

## ML-1m - 20e - 0.2 train/0.8 test

In [2]:
results = [
'../metrics/mfnn/2016-05-13_13.07.07_mfnn_ml-1m_e20_tt-0.2_task-19_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.07.24_mfnn_ml-1m_e20_tt-0.2_task-18_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.08.14_mfnn_ml-1m_e20_tt-0.2_task-3_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.08.42_mfnn_ml-1m_e20_tt-0.2_task-11_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.08.49_mfnn_ml-1m_e20_tt-0.2_task-5_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.08.59_mfnn_ml-1m_e20_tt-0.2_task-1_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.13.32_mfnn_ml-1m_e20_tt-0.2_task-0_user-metrics.csv',
'../metrics/mfnn/2016-05-13_14.02.05_mfnn_ml-1m_e20_tt-0.2_task-7_user-metrics.csv',
'../metrics/mfnn/2016-05-13_14.19.45_mfnn_ml-1m_e20_tt-0.2_task-13_user-metrics.csv',
'../metrics/mfnn/2016-05-13_14.22.53_mfnn_ml-1m_e20_tt-0.2_task-12_user-metrics.csv',
'../metrics/mfnn/2016-05-13_14.54.52_mfnn_ml-1m_e20_tt-0.2_task-17_user-metrics.csv',
'../metrics/mfnn/2016-05-13_16.17.19_mfnn_ml-1m_e20_tt-0.2_task-6_user-metrics.csv',
'../metrics/mfnn/2016-05-13_16.30.09_mfnn_ml-1m_e20_tt-0.2_task-20_user-metrics.csv',
'../metrics/mfnn/2016-05-13_16.36.33_mfnn_ml-1m_e20_tt-0.2_task-14_user-metrics.csv',
'../metrics/mfnn/2016-05-13_17.12.47_mfnn_ml-1m_e20_tt-0.2_task-21_user-metrics.csv',
'../metrics/mfnn/2016-05-13_17.53.02_mfnn_ml-1m_e20_tt-0.2_task-8_user-metrics.csv',
'../metrics/mfnn/2016-05-13_18.35.07_mfnn_ml-1m_e20_tt-0.2_task-23_user-metrics.csv',
'../metrics/mfnn/2016-05-13_19.30.42_mfnn_ml-1m_e20_tt-0.2_task-10_user-metrics.csv',
'../metrics/mfnn/2016-05-13_20.08.12_mfnn_ml-1m_e20_tt-0.2_task-22_user-metrics.csv',
'../metrics/mfnn/2016-05-13_20.10.52_mfnn_ml-1m_e20_tt-0.2_task-15_user-metrics.csv',
'../metrics/mfnn/2016-05-13_22.18.50_mfnn_ml-1m_e20_tt-0.2_task-2_user-metrics.csv',
]

In [3]:
result_map = map(lambda p: (p, pd.read_csv(p)['auc'].mean()), results)
sorted(result_map, key=lambda t:t[1], reverse=True)

[('../metrics/mfnn/2016-05-13_16.17.19_mfnn_ml-1m_e20_tt-0.2_task-6_user-metrics.csv',
  0.8978495667178319),
 ('../metrics/mfnn/2016-05-13_19.30.42_mfnn_ml-1m_e20_tt-0.2_task-10_user-metrics.csv',
  0.8911205847799619),
 ('../metrics/mfnn/2016-05-13_13.08.49_mfnn_ml-1m_e20_tt-0.2_task-5_user-metrics.csv',
  0.8889464453164606),
 ('../metrics/mfnn/2016-05-13_13.08.42_mfnn_ml-1m_e20_tt-0.2_task-11_user-metrics.csv',
  0.8830935640725851),
 ('../metrics/mfnn/2016-05-13_13.13.32_mfnn_ml-1m_e20_tt-0.2_task-0_user-metrics.csv',
  0.8804266743776397),
 ('../metrics/mfnn/2016-05-13_22.18.50_mfnn_ml-1m_e20_tt-0.2_task-2_user-metrics.csv',
  0.8781281670501629),
 ('../metrics/mfnn/2016-05-13_20.10.52_mfnn_ml-1m_e20_tt-0.2_task-15_user-metrics.csv',
  0.8779493811055779),
 ('../metrics/mfnn/2016-05-13_14.19.45_mfnn_ml-1m_e20_tt-0.2_task-13_user-metrics.csv',
  0.8776865504778436),
 ('../metrics/mfnn/2016-05-13_17.53.02_mfnn_ml-1m_e20_tt-0.2_task-8_user-metrics.csv',
  0.8774628776136015),
 ('../

In [4]:
# best config
with open('../models/mfnn/2016-05-13_16.17.19_mfnn_ml-1m_e20_tt-0.2_task-6_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': False,
    u'experiment_name': u'mfnn_ml-1m_e20_tt-0.2_task-6',
    u'hit_threshold': 4,
    u'lr': 0.01,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 96,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'reg_lambda': 0.01,
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.2-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.2-train.csv',
    u'train_test_split': 0.2,
    u'use_avg_rating': False,
    u'user

In [5]:
with open('../models/mfnn/2016-05-13_19.30.42_mfnn_ml-1m_e20_tt-0.2_task-10_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': False,
    u'experiment_name': u'mfnn_ml-1m_e20_tt-0.2_task-10',
    u'hit_threshold': 4,
    u'lr': 0.03,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 128,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'reg_lambda': 0.01,
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.2-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.2-train.csv',
    u'train_test_split': 0.2,
    u'use_avg_rating': True,
    u'use

In [6]:
with open('../models/mfnn/2016-05-13_13.08.49_mfnn_ml-1m_e20_tt-0.2_task-5_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': False,
    u'experiment_name': u'mfnn_ml-1m_e20_tt-0.2_task-5',
    u'hit_threshold': 4,
    u'lr': 0.01,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 64,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'reg_lambda': 0.003,
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.2-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.2-train.csv',
    u'train_test_split': 0.2,
    u'use_avg_rating': False,
    u'use